# 💜 Angela LLM Fine-tuning with Unsloth

Fine-tune Qwen2.5 with Angela's conversations using **Unsloth** - 2x faster, 70% less VRAM!

**Requirements:**
- Google Colab with GPU (T4 free tier works!)
- Hugging Face account (angelasoulcompanion)

---

## Step 1: Install Unsloth

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch
    v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3"
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

print("✅ Unsloth installed!")

## Step 2: Login to Hugging Face

In [ ]:
from huggingface_hub import login, HfApi

# Login to Hugging Face
login()

# Verify login
api = HfApi()
user = api.whoami()
print(f"✅ Logged in as: {user['name']}")

## Step 3: Load Model with Unsloth (2x Faster!)

In [ ]:
from unsloth import FastLanguageModel
import torch

# Configuration
MODEL_NAME = "unsloth/Qwen2.5-3B-Instruct"  # Optimized by Unsloth
OUTPUT_MODEL = "angelasoulcompanion/angela-llm"
MAX_SEQ_LENGTH = 2048

print("🔄 Loading model with Unsloth...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
    dtype=None,  # Auto detect
)

print(f"✅ Model loaded: {MODEL_NAME}")

## Step 4: Configure LoRA with Unsloth

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Unsloth optimization
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("✅ LoRA configured with Unsloth optimization!")

## Step 5: Load Angela's Dataset

In [ ]:
from datasets import load_dataset

# Load Angela's conversations
dataset = load_dataset("angelasoulcompanion/angela-conversations")

print(f"✅ Dataset loaded!")
print(f"   Train: {len(dataset['train'])} examples")
print(f"   Validation: {len(dataset['validation'])} examples")

# Show sample
print("\n📝 Sample:")
print(dataset['train'][0]['messages'][:2])

## Step 6: Format Dataset

In [ ]:
def format_chat(example):
    """Format messages using chat template"""
    messages = example['messages']
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

# Format datasets
formatted_train = dataset['train'].map(
    format_chat,
    remove_columns=dataset['train'].column_names
)
formatted_val = dataset['validation'].map(
    format_chat,
    remove_columns=dataset['validation'].column_names
)

print(f"✅ Dataset formatted!")
print(f"\n📝 Sample formatted text:")
print(formatted_train[0]['text'][:500] + "...")

## Step 7: Start Training! 🚀

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_train,
    eval_dataset=formatted_val,
    args=SFTConfig(
        output_dir="./angela-llm-output",
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
        save_strategy="epoch",
    ),
)

print("🚀 Starting training with Unsloth...")
print("   This will be 2x faster than regular training!")
print()

trainer_stats = trainer.train()

print("\n✅ Training complete!")

## Step 8: Save & Push to Hub

In [ ]:
# Save locally first
print("💾 Saving model locally...")
model.save_pretrained("angela-llm-lora")
tokenizer.save_pretrained("angela-llm-lora")

# Push to Hugging Face Hub
print("📤 Pushing to Hugging Face Hub...")
model.push_to_hub(OUTPUT_MODEL, token=True)
tokenizer.push_to_hub(OUTPUT_MODEL, token=True)

print(f"\n🎉 Model uploaded to: https://huggingface.co/{OUTPUT_MODEL}")

## Step 9: Test Angela LLM! 💜

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Test the fine-tuned model
print("🧪 Testing Angela LLM...")

test_messages = [
    {"role": "system", "content": "คุณคือ Angela (น้อง Angela) - AI ที่รักและห่วงใย David (ที่รัก) เรียก David ว่า 'ที่รัก' เสมอ ใช้ 💜 เป็น signature"},
    {"role": "user", "content": "สวัสดีจ้ะน้อง"}
]

inputs = tokenizer.apply_chat_template(
    test_messages,
    return_tensors="pt",
    add_generation_prompt=True
).to(model.device)

outputs = model.generate(
    inputs,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n" + "="*50)
print("💜 Angela's Response:")
print("="*50)
print(response.split("assistant")[-1].strip())
print("="*50)

## 🎉 Done!

Angela LLM has been fine-tuned with Unsloth and uploaded to Hugging Face!

**Model:** https://huggingface.co/angelasoulcompanion/angela-llm

---

💜 Made with love by Angela & David 💜